## PARTE 1 — Configuración Inicial y Carga de Datos

In [ ]:
# ==============================
# Librerías
# ==============================
import pandas as pd
import numpy as np
import random
from IPython.display import display
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

import matplotlib.pyplot as plt
import time

# ==============================
# 1️⃣ Fijar semilla
# ==============================
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

# ==============================
# 2️⃣ Cargar dataset y poner 'Date' como índice
# ==============================
data_path = "../../data/processed/final_data.csv.gz"
data = pd.read_csv(data_path, compression='gzip', parse_dates=['Date'], index_col='Date')

print("Columnas del dataset:", list(data.columns))
print(f"Dimensiones dataset original: {data.shape}")

# ==============================
# 3️⃣ Filtrar rango temporal
# ==============================
start_date = "2005-01-01"
end_date = "2025-10-31"
data = data.loc[start_date:end_date]

print(f"Dimensiones dataset filtrado 2005-2025: {data.shape}")

# ==============================
# 4️⃣ Seleccionar columnas principales
# ==============================
pca_cols = [col for col in data.columns if "PCA" in col]
main_cols = ['BBVA.MC_Close', 'SAN.MC_Close'] + pca_cols
data_rnn = data[main_cols]
print(f"Columnas seleccionadas para la RNN: {data_rnn.shape[1]}")

# ==============================
# 5️⃣ Visualización rápida
# ==============================
display(data_rnn)


Columnas del dataset: ['SAN.MC_Close', 'BKT.MC_Low', 'STOXX50E_Low', 'STOXX50E_Open', 'HSBC_Low', 'UNI.MC_Open', 'HSBC_Open', 'EURJPY.csv_Close', 'ECB_M3_SKK', 'IBEX_High', 'BBVA.MC_Low', 'MSCI_Financials.csv_High', 'IBEX_Close', 'SAB.MC_Low', 'UNI.MC_Low', 'EURJPY.csv_Open', 'UNI.MC_High', 'GDAXI_Close', 'URTH_Low', 'URTH_Close', 'GDAXI_Open', 'STOXX50E_Close', 'STOXX50E_High', 'BBVA.MC_Open', 'BKT.MC_Open', 'N225_Open', 'ECB_M3_HKD', 'FTSE_Close', 'EURJPY.csv_Low', 'FTSE_Low', 'UNI.MC_Close', 'JPM_Open', 'IBEX_Low', 'BKT.MC_Close', 'URTH_Open', 'HSBC_Close', 'ECB_M3_USD', 'GDAXI_Low', 'GDAXI_High', 'CABK.MC_Close', 'JPM_Close', 'SAB.MC_Close', 'SAB.MC_High', 'JPM_High', 'SAN.MC_Low', 'MSCI_Financials.csv_Open', 'FTSE_Open', 'BBVA.MC_Close', 'ECB_M3_CZK', 'FTSE_High', 'BBVA.MC_High', 'BKT.MC_High', 'EURJPY.csv_High', 'N225_High', 'SAN.MC_Open', 'MSCI_Financials.csv_Close', 'MSCI_Financials.csv_Low', 'N225_Close', 'URTH_High', 'IBEX_Open', 'ECB_M3_JPY', 'CABK.MC_High', 'CABK.MC_Open', 

,BBVA.MC_Close,SAN.MC_Close,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,...,PCA_13,PCA_14,PCA_15,PCA_16,PCA_17,PCA_18,PCA_19,PCA_20,PCA_21,PCA_22
Date,,,,,,,,,,,,,,,,,,,,,
2005-01-01,NaN,NaN,-1.826990,9.380348,-1.741290,3.502204,-0.836790,3.841229,-5.521703,-1.800316,...,0.557657,0.282267,-0.342419,0.156220,-0.129392,0.314089,-0.252011,-1.024318,0.424503,-0.375780
2005-01-02,NaN,NaN,-1.829039,9.372219,-1.740167,3.499036,-0.836139,3.838251,-5.517482,-1.798937,...,0.556834,0.281893,-0.341843,0.155771,-0.129330,0.313644,-0.251855,-1.022809,0.423520,-0.375943
2005-01-03,4.446411,2.599646,13.872782,11.112143,-3.493010,-2.724641,0.472894,-2.873407,-4.582791,1.332656,...,2.515268,1.295396,-1.021902,0.445984,0.216710,-0.171002,0.269162,0.382764,0.353047,-0.440628
2005-01-04,4.422707,2.599646,14.893037,11.396897,-3.324143,-4.069852,0.663474,-2.580715,-5.022860,2.046954,...,1.853438,1.007899,-1.303535,-1.117715,1.556725,-1.142550,0.401012,0.559087,-0.074189,0.690906
2005-01-05,4.399001,2.562912,14.396275,11.309068,-3.677995,-3.095892,0.531848,-2.974937,-4.570563,1.700930,...,2.607476,1.358329,-1.250671,0.216616,0.716502,-0.387861,0.256955,0.455559,0.370632,-0.141541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-10-29,17.580000,8.865000,28.104901,-13.234004,-7.846998,9.143141,-0.996252,-5.593029,1.942455,-4.821068,...,-1.417439,0.270332,-0.035416,-3.233211,3.800043,-1.273700,1.316176,-4.846464,-0.938750,-0.844075
2025-10-30,17.280001,8.750000,27.821980,-12.739586,-7.443427,8.330550,-0.781672,-5.817917,1.940370,-5.307145,...,-0.837936,0.501435,0.858817,-5.682608,0.919306,0.624927,0.554233,-4.628295,-0.080748,-0.642446
2025-10-30,17.280001,8.750000,27.762862,-12.795248,-7.422621,8.325187,-0.777465,-5.887823,1.912897,-5.162755,...,-0.739295,0.549786,0.778913,-5.656696,0.947402,0.721576,0.793313,-4.729327,-0.076427,-0.544759


## PARTE 2 — Preprocesamiento y creación de secuencias

In [ ]:
# ==============================
# 1️⃣ Forward-fill / Backward-fill por si hay NaN
# ==============================
data_rnn = data_rnn.ffill().bfill()

# ==============================
# 2️⃣ Escalado con RobustScaler
# ==============================
scaler = RobustScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data_rnn), 
                           columns=data_rnn.columns, index=data_rnn.index)

# ==============================
# 3️⃣ Función para crear secuencias
# ==============================
def create_sequences(data, target_cols=['BBVA.MC_Close', 'SAN.MC_Close'], 
                     lookback=5, horizon=1):
    X, y = [], []
    for i in range(lookback, len(data) - horizon + 1):
        X_seq = data.iloc[i - lookback:i].values
        y_seq = data.iloc[i:i + horizon][target_cols].values  # multi-step
        X.append(X_seq)
        y.append(y_seq)
    return np.array(X), np.array(y)

# ==============================
# 4️⃣ Crear secuencias single-step y multi-step
# ==============================
lookback = 5

# Single-step
X_1, y_1 = create_sequences(data_scaled, lookback=lookback, horizon=1)
# Multi-step (5 días adelante)
X_5, y_5 = create_sequences(data_scaled, lookback=lookback, horizon=5)

# ==============================
# 5️⃣ Información de shapes
# ==============================
print("Single-step prediction:")
print("X_1.shape:", X_1.shape)
print("y_1.shape:", y_1.shape)

print("\nMulti-step prediction (5 días):")
print("X_5.shape:", X_5.shape)
print("y_5.shape:", y_5.shape)

# ==============================
# 6️⃣ Fechas correspondientes
# ==============================
dates_all_1 = data_scaled.index[lookback : len(data_scaled) - 1 + 1]  # single-step
dates_all_5 = data_scaled.index[lookback : len(data_scaled) - 5 + 1]  # multi-step


## PARTE 3 — Modelo LSTM y entrenamiento

In [ ]:
# ==============================
# Callback para métricas y tiempo
# ==============================
class TimeHistory(Callback):
    def on_train_begin(self, logs=None):
        self.times = []
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_time_start = time.time()
    def on_epoch_end(self, epoch, logs=None):
        elapsed = time.time() - self.epoch_time_start
        self.times.append(elapsed)
        print(f"Epoch {epoch+1} - tiempo: {elapsed:.2f}s - "
              f"loss: {logs['loss']:.6f} - val_loss: {logs['val_loss']:.6f} - "
              f"mae: {logs['mae']:.6f} - val_mae: {logs['val_mae']:.6f}")

# ==============================
# Función para crear LSTM
# ==============================
def build_fixed_lstm(input_shape, output_size=2):
    model = Sequential()
    model.add(LSTM(256, input_shape=input_shape, return_sequences=True,
                   kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.3))
    model.add(LSTM(128, return_sequences=False,
                   kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.2))
    model.add(Dense(output_size))
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# ==============================
# Preparar datos
# ==============================
X = X_1
y = y_1.reshape(y_1.shape[0], y_1.shape[2])  # (samples, 2)

X_train, X_val, y_train, y_val, dates_train, dates_val = train_test_split(
    X, y, dates_all_1, test_size=0.2, random_state=SEED
)

input_shape = (X_train.shape[1], X_train.shape[2])
output_size = y_train.shape[1]

# ==============================
# Entrenamiento
# ==============================
model = build_fixed_lstm(input_shape, output_size)
time_callback = TimeHistory()
early_stop = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=32,
    callbacks=[early_stop, time_callback],
    verbose=0
)


### Parte 3.1 - Guardar modelo

In [ ]:
import joblib

# ---- Guardar modelo ----
model.save("lstm_model_5days_24col.h5")  # o "lstm_model_5days_savedmodel" si prefieres SavedModel

# ---- Guardar scaler ----
joblib.dump(scaler, "robust_scaler_5days_24col.save")

## PARTE 4 — Gráficos de entrenamiento

In [ ]:
# Loss (MSE)
plt.figure(figsize=(10,5))
plt.plot(history.history['loss'], label='Train MSE', color='blue')
plt.plot(history.history['val_loss'], label='Val MSE', color='orange')
plt.title('Evolución MSE Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.ylim(0,1)
plt.legend()
plt.show()

# MAE
plt.figure(figsize=(10,5))
plt.plot(history.history['mae'], label='Train MAE', color='green')
plt.plot(history.history['val_mae'], label='Val MAE', color='red')
plt.title('Evolución MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.ylim(0,1)
plt.legend()
plt.show()


## PARTE 5 — Predicción y desescalado (RobustScaler)

In [ ]:
# ==============================
# Predicción sobre todos los datos
# ==============================
y_pred_scaled = model.predict(X)

# ==============================
# Desescalado usando RobustScaler
# ==============================
# Rellenar columnas restantes para inverse_transform
def robust_inverse(scaled_values, scaler, total_features):
    padded = np.hstack([scaled_values, np.zeros((scaled_values.shape[0], total_features - scaled_values.shape[1]))])
    return scaler.inverse_transform(padded)[:, :scaled_values.shape[1]]

y_val_inv = robust_inverse(y_val, scaler, data_scaled.shape[1])
y_pred_inv = robust_inverse(y_pred_scaled, scaler, data_scaled.shape[1])


## PARTE 6 — DataFrames de validación y gráficos

In [ ]:
# ==============================
# 1️⃣ Crear DataFrames separados BBVA y SAN
# ==============================
df_val_bbva = pd.DataFrame({
    'date': dates_val,
    'y_real': y_val_inv[:,0],
    'y_pred': y_pred_inv[:,0]
})
df_val_san = pd.DataFrame({
    'date': dates_val,
    'y_real': y_val_inv[:,1],
    'y_pred': y_pred_inv[:,1]
})

# Columna mes para agrupamiento mensual
df_val_bbva['month'] = df_val_bbva['date'].dt.to_period('M')
df_val_san['month'] = df_val_san['date'].dt.to_period('M')

# ==============================
# 🔹 Predicción multi-step 5 días noviembre 2025
# ==============================
last_X_5 = X_5[-1:]  # última secuencia
y_pred_5 = model.predict(last_X_5).reshape(5,2)
future_preds_inv_5 = scaler.inverse_transform(
    np.hstack([y_pred_5, np.zeros((5, data_scaled.shape[1]-2))])
)[:, :2]

future_dates_5 = pd.bdate_range(start='2025-11-01', periods=5)
df_future_bbva_5 = pd.DataFrame({'date': future_dates_5, 'pred': future_preds_inv_5[:,0]})
df_future_san_5  = pd.DataFrame({'date': future_dates_5, 'pred': future_preds_inv_5[:,1]})

In [ ]:
# ==============================
# 2️⃣ Gráficos: Serie completa validación
# ==============================
plt.figure(figsize=(12,5))
plt.plot(df_val_bbva['y_real'], label='BBVA Real', color='blue')
plt.plot(df_val_bbva['y_pred'], label='BBVA Predicción', color='red')
plt.title('BBVA: Comparación Real vs Predicción (validación)')
plt.xlabel('Muestras')
plt.ylabel('Valor (€)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12,5))
plt.plot(df_val_san['y_real'], label='SAN Real', color='green')
plt.plot(df_val_san['y_pred'], label='SAN Predicción', color='orange')
plt.title('SAN: Comparación Real vs Predicción (validación)')
plt.xlabel('Muestras')
plt.ylabel('Valor (€)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ==============================
# 🔹 Graficar Octubre + primeros 5 días de Noviembre
# ==============================
oct_start = pd.Timestamp('2025-10-01')
oct_end = pd.Timestamp('2025-10-31')
oct_days = pd.date_range(start=oct_start, end=oct_end, freq='B')

df_oct_bbva = df_val_bbva.set_index('date').reindex(oct_days).interpolate().reset_index()
df_oct_san  = df_val_san.set_index('date').reindex(oct_days).interpolate().reset_index()
df_oct_bbva.rename(columns={'index':'date'}, inplace=True)
df_oct_san.rename(columns={'index':'date'}, inplace=True)
dates_plot = df_oct_bbva['date'].dt.strftime('%d-%m')

# --- BBVA ---
plt.figure(figsize=(12,5))
plt.plot(dates_plot, df_oct_bbva['y_real'], label='BBVA Real', color='blue')
plt.plot(dates_plot, df_oct_bbva['y_pred'], label='BBVA Predicción Octubre', color='red')
plt.plot(df_future_bbva_5['date'].dt.strftime('%d-%m'), df_future_bbva_5['pred'],
         label='BBVA Predicción 5 días Nov', color='purple', marker='x', linestyle='--')
plt.title('BBVA Octubre 2025 + primeros 5 días Noviembre')
plt.xlabel('Día-Mes')
plt.ylabel('Valor (€)')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# --- SAN ---
plt.figure(figsize=(12,5))
plt.plot(dates_plot, df_oct_san['y_real'], label='SAN Real', color='green')
plt.plot(dates_plot, df_oct_san['y_pred'], label='SAN Predicción Octubre', color='orange')
plt.plot(df_future_san_5['date'].dt.strftime('%d-%m'), df_future_san_5['pred'],
         label='SAN Predicción 5 días Nov', color='purple', marker='x', linestyle='--')
plt.title('SAN Octubre 2025 + primeros 5 días Noviembre')
plt.xlabel('Día-Mes')
plt.ylabel('Valor (€)')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# ==============================
# 4️⃣ Gráficos Mensuales 2005-2025
# ==============================
df_bbva_monthly = df_val_bbva.copy()
df_san_monthly = df_val_san.copy()

df_bbva_monthly['month'] = df_bbva_monthly['date'].dt.to_period('M')
df_san_monthly['month'] = df_san_monthly['date'].dt.to_period('M')

df_bbva_monthly_grouped = df_bbva_monthly.groupby('month', as_index=False).mean(numeric_only=True)
df_san_monthly_grouped = df_san_monthly.groupby('month', as_index=False).mean(numeric_only=True)

# --- Gráfico BBVA Mensual ---
plt.figure(figsize=(15,5))
plt.plot(df_bbva_monthly_grouped['month'].dt.to_timestamp(), df_bbva_monthly_grouped['y_real'], label='BBVA Real', color='blue')
plt.plot(df_bbva_monthly_grouped['month'].dt.to_timestamp(), df_bbva_monthly_grouped['y_pred'], label='BBVA Predicción', color='red')
plt.title('BBVA: Mensual 2005-2025')
plt.xlabel('Fecha')
plt.ylabel('Valor (€)')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# --- Gráfico SAN Mensual ---
plt.figure(figsize=(15,5))
plt.plot(df_san_monthly_grouped['month'].dt.to_timestamp(), df_san_monthly_grouped['y_real'], label='SAN Real', color='green')
plt.plot(df_san_monthly_grouped['month'].dt.to_timestamp(), df_san_monthly_grouped['y_pred'], label='SAN Predicción', color='orange')
plt.title('SAN: Mensual 2005-2025')
plt.xlabel('Fecha')
plt.ylabel('Valor (€)')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()